# Regional concentration index

The regional concentration index (RCI) measures the extent to which a collection of countries (a “region”) is trading among themselves relative to the rest of the world. It is computed as a ratio of two fractions: the share of the region’s trade with itself and the share of the region in world trade: 

$$
{RCI}_q = \frac{{Exports}_{qq} + {Imports}_{qq}}{{Exports}_q + {Imports}_q} \Bigg/ \frac{{Exports}_q + {Imports}_q}{{Exports}_{world} + {Imports}_{world}}
$$

The results of this notebook are saved as `flows.parquet` in `data/interim/` and `rci.parquet` in `data/final`.

In [168]:
import numpy as np
import pandas as pd
import os
import re
import duckdb
from functions import asvector

## Set up

In [169]:
inputfolder = 'ADB-MRIO'
output1 = 'flows'
output2 = 'rci'
index = 'mrio'

# inputfolder = 'ADB-MRIO62'
# output1 = 'flows62'
# output2 = 'rci62'
# index = 'mrio62'

filelist = [file for file in os.listdir(f'../data/interim/{inputfolder}') if not file.startswith('.')]
filelist.sort()

countries = pd.read_excel('../data/raw/countries.xlsx')
sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

G = 73      # Number of countries + ROW
# G = 63
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

## Compute flows

In [170]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    B = np.linalg.inv(np.eye(G*N) - A)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    VBY = np.diag(v) @ B @ Y

    DFi = pd.DataFrame({
        't': year,
        's': np.tile(np.arange(1, G+1).repeat(N), G),
        'r': np.arange(1, G+1).repeat(G*N),
        'i': np.tile(sectors['ind'], G*G),
        'i5': np.tile(sectors['ind5'], G*G),
        'i15': np.tile(sectors['ind15'], G*G),
        'flows': asvector(VBY)
    })
    DF = pd.concat([DF, DFi], ignore_index=True)

    print(f'{year} done')

DF.sort_values(['t', 's', 'r', 'i'], inplace=True)
DF.to_parquet(f'../data/interim/{output1}.parquet', index=False)

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [171]:
DF

,t,s,r,i,i5,i15,flows
0,2017,1,1,1,1,1,24580.928164
1,2017,1,1,2,1,2,17033.117412
2,2017,1,1,3,2,3,14724.957952
3,2017,1,1,4,2,3,1059.935367
4,2017,1,1,5,2,3,142.991659
...,...,...,...,...,...,...,...
1119085,2022,73,73,31,5,13,336582.229471
1119086,2022,73,73,32,5,14,245745.094921
1119087,2022,73,73,33,5,14,154924.793812
1119088,2022,73,73,34,5,15,129306.912080


## Compute RCI

In [172]:
flows = duckdb.sql(
    f"""
    SELECT t, s, r, SUM(flows) AS flows
    FROM read_parquet('../data/interim/{output1}.parquet')
    WHERE s <> r
    GROUP BY t, s, r
    ORDER BY t, s, r
    """
).df()

In [173]:
rta = countries[[f'{index}', 'rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta']]

In [174]:
world = flows.groupby(['t'])['flows'].sum().reset_index()
world.rename(columns={'flows': 'world'}, inplace=True)

In [175]:
exports = flows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
exports = exports.melt(
    id_vars=['t', 's', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
exports['rta'] = exports['rta'].str.split('_').str[-1]
exports.dropna(inplace=True)
exports = exports.groupby(['t', 'rta'])['flows'].sum().reset_index()
exports.rename(columns={'flows': 'exports'}, inplace=True)

In [176]:
imports = flows.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
imports = imports.melt(
    id_vars=['t', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='rta',
    value_name='member'
)
imports['rta'] = imports['rta'].str.split('_').str[-1]
imports.dropna(inplace=True)
imports = imports.groupby(['t', 'rta'])['flows'].sum().reset_index()
imports.rename(columns={'flows': 'imports'}, inplace=True)

In [177]:
within = flows.merge(rta, left_on='s', right_on=f'{index}').drop([f'{index}'], axis=1)
within = within.melt(
    id_vars=['t', 's', 'r', 'flows'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='exporter',
    value_name='member'
)
within['exporter'] = within['exporter'].str.split('_').str[-1]
within.dropna(inplace=True)
within.drop(['member'], axis=1, inplace=True)

within = within.merge(rta, left_on='r', right_on=f'{index}').drop([f'{index}'], axis=1)
within = within.melt(
    id_vars=['t', 's', 'r', 'flows', 'exporter'], 
    value_vars=['rta_asean', 'rta_eaeu', 'rta_eu', 'rta_nafta', 'rta_safta'],
    var_name='importer',
    value_name='member'
)
within['importer'] = within['importer'].str.split('_').str[-1]
within.dropna(inplace=True)
within.drop(['member'], axis=1, inplace=True)

within = within.groupby(['t', 'exporter', 'importer'])['flows'].sum().reset_index()
within = within[within['exporter'] == within['importer']]
within.rename(columns={'exporter': 'rta', 'flows': 'within'}, inplace=True)
within.drop(['importer'], axis=1, inplace=True)

In [178]:
rci = pd.merge(within, exports).merge(imports).merge(world)
rci['share_within'] = (2 * rci['within']) / (rci['exports'] + rci['imports'])
rci['share_world'] = (rci['exports'] + rci['imports']) / (2 * rci['world'])
rci['rci'] = rci['share_within'] / rci['share_world']

rci.to_parquet(f'../data/final/{output2}.parquet', index=False)

In [179]:
rci

,t,rta,within,exports,imports,world,share_within,share_world,rci
0,2017,asean,1.107488e+05,8.876582e+05,7.189805e+05,1.624261e+07,0.137864,0.049458,2.787522
1,2017,eaeu,1.873526e+04,4.272941e+05,3.334898e+05,1.624261e+07,0.049253,0.023419,2.103066
2,2017,eu,1.687668e+06,4.323921e+06,3.670099e+06,1.624261e+07,0.422233,0.246082,1.715823
3,2017,nafta,6.617447e+05,2.572996e+06,3.132204e+06,1.624261e+07,0.231979,0.175624,1.320883
4,2017,safta,1.919762e+04,3.976537e+05,4.597695e+05,1.624261e+07,0.044780,0.026394,1.696574
5,2018,asean,1.203666e+05,9.982560e+05,8.604663e+05,1.814395e+07,0.129515,0.051222,2.528534
6,2018,eaeu,2.123891e+04,5.199371e+05,3.374629e+05,1.814395e+07,0.049543,0.023628,2.096800
7,2018,eu,1.947479e+06,5.038497e+06,4.423627e+06,1.814395e+07,0.411637,0.260752,1.578655
8,2018,nafta,7.684108e+05,2.726196e+06,3.393208e+06,1.814395e+07,0.251139,0.168635,1.489248
9,2018,safta,2.385420e+04,4.990190e+05,6.747725e+05,1.814395e+07,0.040645,0.032347,1.256535
